In [22]:
import json
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import time
import os
import logging

# Configure logging
logging.basicConfig(filename='data_collection.log', level=logging.ERROR,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Function to save training vector to a JSON file
def save_training_vector(data, filename):
    try:
        if os.path.exists(filename):
            with open(filename, 'r') as file:
                existing_data = json.load(file)
            existing_data.append(data)
            with open(filename, 'w') as file:
                json.dump(existing_data, file, indent=4)
        else:
            with open(filename, 'w') as file:
                json.dump([data], file, indent=4)
    except Exception as e:
        logging.error(f"Error occurred while saving training vector to {filename}: {e}")

# Function to fetch trade history and order book data from Trade Ogre API
def fetch_data(pair):
    try:
        trade_data = requests.get(f'https://tradeogre.com/api/v1/history/{pair}')
        order_data = requests.get(f'https://tradeogre.com/api/v1/orders/{pair}')
        
        if trade_data.status_code != 200 or order_data.status_code != 200:
            logging.error(f"Failed to fetch data from Trade Ogre API: Status Code - Trade: {trade_data.status_code}, Orders: {order_data.status_code}")
            return None, None
        
        trades = json.loads(trade_data.text)
        orders = json.loads(order_data.text)
        
        return trades, orders
    except Exception as e:
        logging.error(f"Error occurred while fetching data from Trade Ogre API: {e}")
        return None, None

# Initialize a variable to store the current UTC date
current_utc_date = datetime.utcnow().strftime("%Y-%m-%d")
# Initialize a set to keep track of processed trades
processed_trades = set()
while True:
    try:
        # Check if the current date has changed
        if datetime.utcnow().strftime("%Y-%m-%d") != current_utc_date:
            # If the date has changed, update the current date and reset the processed trades set
            current_utc_date = datetime.utcnow().strftime("%Y-%m-%d")
            processed_trades = set()

        # Fetch trade and order book data
        trades, orders = fetch_data('FLUX-USDT')

        if trades is None or orders is None:
            raise Exception("Failed to fetch data from Trade Ogre API")

        # Calculate VWAP
        prices = np.array([float(trade['price']) for trade in trades])
        volumes = np.array([float(trade['quantity']) for trade in trades])
        vwap = np.average(prices, weights=volumes)

        # Check for new trades
        for trade in trades:
            timestamp = trade['date']
            price = float(trade['price'])
            quantity = float(trade['quantity'])
            trade_id = f"{timestamp}_{price}_{quantity}"  # Concatenate date, price, and quantity
    
            if trade_id not in processed_trades:
                # Extract features from order book (e.g., bid-ask spread)
                buy_prices = list(orders['buy'].keys())
                buy_prices = np.float_(buy_prices)
                sell_prices = list(orders['sell'].keys())
                sell_prices = np.float_(sell_prices)
                bid_ask_spread = buy_prices[0] - sell_prices[0]
        
                # Combine features into a feature vector
                feature_vector = {'timestamp': timestamp, 'price': price, 'qty': quantity,
                                  'vwap': vwap, 'bid_ask_spread': bid_ask_spread}

                # Generate filename with UTC date
                filename = f"FLUX-USDT_{current_utc_date}.json"

                # Save training vector to JSON file
                save_training_vector(feature_vector, filename)

                # Mark trade as processed
                processed_trades.add(trade_id)

                print(f"New trade processed and saved to {filename}")


        # Wait for a short interval before checking for new trades again
        time.sleep(2)  # Adjust the interval as needed

    except Exception as e:
        logging.error(f"An error occurred: {e}")
        time.sleep(15)  # Wait before retrying

New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.js

KeyboardInterrupt: 

In [19]:
buy_prices

NameError: name 'buy_prices' is not defined

In [18]:
trade_id

0     1711151121
1     1711151121
2     1711151668
3     1711151771
4     1711152978
         ...    
95    1711247924
96    1711249712
97    1711251928
98    1711252320
99    1711252479
Name: date, Length: 100, dtype: int64

In [10]:
prices = trades['price']

In [12]:
trade_id

NameError: name 'trade_id' is not defined

In [3]:
trades['price']

0     1.279266
1     1.279266
2     1.279266
3     1.279266
4     1.279266
        ...   
95    1.312092
96    1.310000
97    1.310000
98    1.310000
99    1.310000
Name: price, Length: 100, dtype: float64

In [6]:
pd.to_numeric(trades['price'])

0     1.279266
1     1.279266
2     1.279266
3     1.279266
4     1.279266
        ...   
95    1.312092
96    1.310000
97    1.310000
98    1.310000
99    1.310000
Name: price, Length: 100, dtype: float64

In [28]:
trades.iloc[-1]

date        1711249712
type              sell
price             1.31
quantity    0.06170809
Name: 99, dtype: object

In [5]:
trades[-1]

{'date': 1711247924,
 'type': 'buy',
 'price': '1.31000009',
 'quantity': '1.33387850'}

In [29]:
import json
import requests
import numpy as np
from datetime import datetime
import time
import os
import logging

# Configure logging
logging.basicConfig(filename='data_collection.log', level=logging.ERROR,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Function to save training vector to a JSON file
def save_training_vector(data, filename):
    try:
        if os.path.exists(filename):
            with open(filename, 'r') as file:
                existing_data = json.load(file)
            existing_data.append(data)
            with open(filename, 'w') as file:
                json.dump(existing_data, file, indent=4)
        else:
            with open(filename, 'w') as file:
                json.dump([data], file, indent=4)
    except Exception as e:
        logging.error(f"Error occurred while saving training vector to {filename}: {e}")

# Function to fetch trade history and order book data from Trade Ogre API
def fetch_data(pair):
    try:
        trade_data = requests.get(f'https://tradeogre.com/api/v1/history/{pair}')
        order_data = requests.get(f'https://tradeogre.com/api/v1/orders/{pair}')
        
        if trade_data.status_code != 200 or order_data.status_code != 200:
            logging.error(f"Failed to fetch data from Trade Ogre API: Status Code - Trade: {trade_data.status_code}, Orders: {order_data.status_code}")
            return None, None
        
        trades = json.loads(trade_data.text)
        orders = json.loads(order_data.text)
        
        return trades, orders
    except Exception as e:
        logging.error(f"Error occurred while fetching data from Trade Ogre API: {e}")
        return None, None

# Initialize a variable to store the current UTC date
current_utc_date = datetime.utcnow().strftime("%Y-%m-%d")
# Initialize a set to keep track of processed trades
processed_trades = set()
while True:
    try:
        # Check if the current date has changed
        if datetime.utcnow().strftime("%Y-%m-%d") != current_utc_date:
            # If the date has changed, update the current date and reset the processed trades set
            current_utc_date = datetime.utcnow().strftime("%Y-%m-%d")
            processed_trades = set()

        # Fetch trade and order book data
        trades, orders = fetch_data('FLUX-USDT')

        if trades is None or orders is None:
            raise Exception("Failed to fetch data from Trade Ogre API")

        # Calculate VWAP
        prices = np.array([float(trade['price']) for trade in trades])
        volumes = np.array([float(trade['quantity']) for trade in trades])
        vwap = np.average(prices, weights=volumes)

        # Check for new trades
        for trade in trades:
            trade_id = trade['date']
            if trade_id not in processed_trades:
                
                # Extract features from order book (e.g., bid-ask spread)
                buy_prices = list(orders['buy'].keys())
                buy_prices = np.float_(buy_prices)
                sell_prices = list(orders['sell'].keys())
                sell_prices = np.float_(sell_prices)
                bid_ask_spread = buy_prices[0] - sell_prices[0]
                
                # Extract features from trade history
                timestamp = trade['date']
                price = float(trade['price'])
                quantity = float(trade['quantity'])
        
                # Combine features into a feature vector
                feature_vector = {'timestamp': timestamp, 'price': price, 'qty': quantity,
                                  'vwap': vwap, 'bid_ask_spread': bid_ask_spread}

                # Generate filename with UTC date
                filename = f"FLUX-USDT_{current_utc_date}.json"

                # Save training vector to JSON file
                save_training_vector(feature_vector, filename)

                # Mark trade as processed
                processed_trades.add(trade_id)

                print(f"New trade processed and saved to {filename}")

        # Wait for a short interval before checking for new trades again
        time.sleep(15)  # Adjust the interval as needed

    except Exception as e:
        logging.error(f"An error occurred: {e}")
        time.sleep(15)  # Wait before retrying

New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.json
New trade processed and saved to FLUX-USDT_2024-03-24.js

KeyboardInterrupt: 

In [79]:
trades[:1]

[{'date': 1711136054,
  'type': 'sell',
  'price': '1.24339283',
  'quantity': '9.09500000'}]

In [82]:
trades[:1]

[{'date': 1711136054,
  'type': 'sell',
  'price': '1.24339283',
  'quantity': '9.09500000'}]

In [ ]:
# Function to save training vector to a JSON file
def save_training_vector(data, filename):
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)

# Function to get current UTC date as a string
def get_utc_date():
    return datetime.utcnow().strftime("%Y-%m-%d")

while True:
    # Check if the current UTC date is different from the last file created
    current_utc_date = get_utc_date()
    if current_utc_date != last_utc_date:
        filename = f"training_data_{current_utc_date}.json"
        last_utc_date = current_utc_date

    # Fetch trade and order book data
    trades, orders = fetch_data('FLUX-USDT')

    # Calculate VWAP
    vwap = calculate_vwap(trades)

    # Check for new trades
    for trade in trades:
        trade_id = trade['date']
        if trade_id not in processed_trades:
            
            # Extract features from order book (e.g., bid-ask spread)
            buy_prices = list(orders['buy'].keys())
            buy_prices = np.float_(buy_prices)
            sell_prices = list(orders['sell'].keys())
            sell_prices = np.float_(sell_prices)
            bid_ask_spread = buy_prices[0] - sell_prices[0]
            
            # Extract features from trade history
            timestamp = trade['date']
            price = float(trade['price'])
            quantity = float(trade['quantity'])
    
            # Current timestamp
            current_t = round(datetime.now().timestamp())

            # Combine features into a feature vector
            feature_vector = {'timestamp': timestamp, 'price': price, 'qty': quantity,
                              'vwap': vwap, 'bid_ask_spread': bid_ask_spread}

            # Save training vector to JSON file
            save_training_vector(feature_vector, filename)

            # Mark trade as processed
            processed_trades.add(trade_id)

            print(f"New trade processed and saved to {filename}")

    # Wait for a short interval before checking for new trades again
    time.sleep(15)  # Adjust the interval as needed
